In [8]:
import pandas as pd, csv, sqlite3

conn = sqlite3.connect('shark_info.db')
c = conn.cursor()
df = pd.read_csv("Mock_data.csv", low_memory=False)
df.to_sql(name='mockup_data', con=conn, if_exists='append', index=False)

conn.commit()